In [ ]:
import pandas as pd
import pylab as pl


pl.style.use('bmh')
pd.options.mode.chained_assignment = None


T=pd.read_csv(r'C:\Users\julia\Documents\python36\datasource\data\revenus_departements.csv')
T.head()

# Repartition in 77

In [ ]:
D=T.query('Dept=="77"').copy()
D

In [ ]:
D=D.shift()
D=D.reset_index(drop=True)
D.loc[0, :] = ['', '', 0, 0]
D

# Cumulated 

In [ ]:
D['ECC_f'] = D['Foyers'].cumsum()
D

In [ ]:
D['ECC_r']=D['Mas_rev'].cumsum()
D

In [ ]:
Revenus_moyen = D['Mas_rev'] / D['Foyers']
Revenus_moyen

# Frequency

In [ ]:
F = D['Foyers'].sum()
F

In [ ]:
R = D['Mas_rev'].sum()
R

In [ ]:
D['FCC_f'] = D['ECC_f'] / F
D['FCC_r'] = D['ECC_r'] / R
D

In [ ]:
import pylab as pl 
pl.style.use ("bmh")

D.plot(x ='FCC_f', y = 'FCC_r',

        marker ='o',

        markerfacecolor = 'red',

        color = 'black',
        
        label = 'Courbe de Lorenz',
        
        xticks = [k/100 for k in range (0, 110, 10)],

        yticks = [k/100 for k in range (0, 110, 10)],

        grid = True, figsize = (8, 8))


for k in range (8):

    pl.text (D. loc [k, 'FCC_f'], D. loc[k, 'FCC_r'],
        r'SM_'+str(k)+'$',
        va = 'top', ha = 'left',
        fontsize = 15)

pl.xlabel('FCC Foyers')
pl.ylabel('FCC Revenus')
pl.legend(loc = 'upper left', fontsize = 14)
pl.axhline (y = 0, color = 'gray')
pl.axvline (x = 0, color = 'gray')

In [ ]:
Revenus_moyen[5]*F/R

In [ ]:
Revenus_moyen * F / R

# Inegality

In [ ]:
def trapeze(b, B, h):
    return (b + B) / 2 * h

In [ ]:
X = list(D['FCC_f'])
Y = list(D['FCC_r'])
print('X = ', X, '\n', 'Y=', Y)

In [ ]:
X[1]-X[0],Y[0],Y[1]

In [ ]:
X[2]-X[1],Y[1],Y[2]

In [ ]:
def aire_trapezes(X, Y):
    A = 0
    for k in range(7):
        A = A + trapeze(Y[k], Y[k+1], X[k+1] - X[k])
    return A

In [ ]:
def gini(X, Y):
    return 1 - 2 * aire_trapezes(X, Y)

In [ ]:
gini(D['FCC_f'], D['FCC_r'])

In [ ]:
def GINI(d):
    # Sélection des lignes de T associées à d
    D = T.query('Dept == @d')
    # indexation de la table D
    D=D.shift()
    D=D.reset_index(drop=True)
    D.loc[0, :] = ['','', 0, 0]
    # Colonnets des fréquences cumulées
    D['FCC_f'] = D['Foyers'].cumsum() / D['Foyers'].sum()
    D['FCC_r'] = D['Mas_rev'].cumsum() / D['Mas_rev'].sum()
    # Listes des coordonnées de la courbe de Lorenz
    x = D['FCC_f']
    y = D['FCC_r']
    return gini(x, y)

In [ ]:
GINI('77')

In [ ]:
GINI('2A')

In [ ]:
import geopandas as gp

J=gp.read_file(r'C:\Users\julia\Documents\python36\datasource\data\departements.json')
J.head()

# Gini Department

In [ ]:
metropole = [d for d in J['code'] if '97' not in d]

In [ ]:
M = pd.DataFrame()
for d in metropole:
    g = round(GINI(d), 3)
    M.loc[d,'gini'] = g
    
    texte = 'Dept ' + d + '<br>' + 'Gini = ' + str(g)
    M.loc[d, 'texte'] = texte
    
M.head()

In [ ]:
M['tranches'] = pd.qcut(x= M['gini'],
                       q =[0, 0.1, 0.25, 0.5, 0.75, 0.9, 1],
                       labels = ['m-D1','D1-Q1','Q1-Me','Me-Q3','Q3-D9','D9-M']
                    
                       )
M.head()

In [ ]:
M.sort_values('gini', inplace = True)
M.head()

In [ ]:
M.index

In [ ]:
import plotly.express as px

fig = px.choropleth(
    data_frame = M,
    locations = M.index,
    geojson=J,
    featureidkey='properties.code',
    color=M['tranches'],
    hover_name=M['texte'],
    color_discrete_sequence=['darkgreen', 'green', 'yellowgreen', 'coral', 'red', 'black'],
    projection='orthographic'
                )

fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(
    margin={'r': 0, 't': 0, 'l': 0, 'b': 0},
    font={'family': 'Arial', 'size': 14, 'color': 'black'
         },
    
    legend={'x': 0.1,
            'y': 0.4, 
            'title': 'Indice de Gini',
            'bordercolor' :'gray',
            'font' : {'family': 'comic sans MS', 'size': 14},
            'borderwidth' : 2,
            'traceorder' :'reversed'
            },
    width=700,
    title={'text': "<b>Indices de Gini des revenus de 2017</b>",
           'font': {'family': 'comic sans MS', 'color': 'black'},
           'y': 0.95,
          }
)
fig.show()